In [4]:
"""
Created on Sat May 5  2021

@author: Hala Hamdoun
"""

'\nCreated on Sat May 5  2021\n\n@author: Hala Hamdoun\n'

In [5]:
import pandas as pd
import numpy as np
from numpy import *
import tensorflow as tf
import os
import random as rn
import sys 
from numpy.random import seed
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from pandas import Series
from pandas import concat
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  app.launch_new_instance()


In [6]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from math import sqrt
import numpy 
from numpy import concatenate

In [7]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['PYTHONHASHSEED'] = '0'
numpy.random.seed(42)
rn.seed(12345)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

In [8]:
from tensorflow.python.keras import backend as k
#tf.set_random_seed(1234)
tf.random.set_seed(1234)

In [9]:
sess =  tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [10]:
def inverse_difference(last_ob, value):
	return value + last_ob

In [11]:
"""**Checking Error** """
from sklearn.metrics import mean_absolute_error
def MAPE(x,y):
	result=0
	for i in range(len(x)):
		result += abs((x[i]-y[i])/x[i])
	result /= len(x)
	result *= 100
	return result

In [12]:
def RMSE(x,y):
	result=0
	for i in range(len(x)):
		result += ((x[i]-y[i])/x[i])**2
	result /= len(x)
	#result = sqrt(result)

	return result

In [13]:
def dRMSE(y_true, y_pred):
	dy_true=np.diff(y_true, axis=0)
	dy_pred=np.diff(y_pred, axis=0)
	result=0
	for i in range(len(dy_true)):
		result += ((dy_true[i]-dy_pred[i])/dy_true[i])**2
	result /= len(dy_true)
	result = sqrt(result)
	return result

In [14]:
df=pd.read_csv('australia.csv')

In [15]:
daily_df=df.iloc[:,0:56]

In [16]:
diff3=daily_df.diff()
#diff3=diff3.values

In [17]:
n_past = 1
n_future = 1
n_features = 56

In [18]:
"""Train - Test Split"""

'Train - Test Split'

In [19]:
train_df,test_df = diff3[0:12], diff3[12:]  
train_df.shape,test_df.shape

((12, 56), (24, 56))

In [20]:
"""Scaling the values for faster training of the models."""

'Scaling the values for faster training of the models.'

In [21]:
train = train_df
scalers={}

In [22]:
for i in train_df.columns:

    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [23]:
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [24]:
"""**Converting the series to samples for supervised learning**"""

'**Converting the series to samples for supervised learning**'

In [25]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [26]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
where_are_NaNs = isnan(X_train)
X_train[where_are_NaNs] = 0
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [27]:
rawdataX,rawdatay=split_series(daily_df.values,n_past, n_future)
rawdatay = rawdatay[12:, :]

In [28]:
model = Sequential()

In [29]:
#model.add(LSTM(15, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
model.add(LSTM(10, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
#model.add(LSTM(7, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
model.add(LSTM(7))
model.add(RepeatVector(n_future))
model.add(LSTM(7, return_sequences=True))
#model.add(LSTM(7, return_sequences=True))
model.add(LSTM(10, return_sequences=True))
#model.add(LSTM(15, return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 10)             2680      
                                                                 
 lstm_1 (LSTM)               (None, 7)                 504       
                                                                 
 repeat_vector (RepeatVector  (None, 1, 7)             0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 1, 7)              420       
                                                                 
 lstm_3 (LSTM)               (None, 1, 10)             720       
                                                                 
 time_distributed (TimeDistr  (None, 1, 56)            616       
 ibuted)                                                

In [30]:
""" **Training the models**"""

' **Training the models**'

In [31]:
history_aus=model.fit(X_train,y_train,epochs=300,validation_split=0.33,batch_size=30,verbose=2,shuffle=False)

Epoch 1/300
1/1 - 11s - loss: 0.3392 - val_loss: 0.3660 - 11s/epoch - 11s/step
Epoch 2/300
1/1 - 0s - loss: 0.3381 - val_loss: 0.3648 - 30ms/epoch - 30ms/step
Epoch 3/300
1/1 - 0s - loss: 0.3370 - val_loss: 0.3636 - 33ms/epoch - 33ms/step
Epoch 4/300
1/1 - 0s - loss: 0.3359 - val_loss: 0.3624 - 30ms/epoch - 30ms/step
Epoch 5/300
1/1 - 0s - loss: 0.3348 - val_loss: 0.3613 - 32ms/epoch - 32ms/step
Epoch 6/300
1/1 - 0s - loss: 0.3337 - val_loss: 0.3601 - 30ms/epoch - 30ms/step
Epoch 7/300
1/1 - 0s - loss: 0.3326 - val_loss: 0.3588 - 38ms/epoch - 38ms/step
Epoch 8/300
1/1 - 0s - loss: 0.3315 - val_loss: 0.3576 - 31ms/epoch - 31ms/step
Epoch 9/300
1/1 - 0s - loss: 0.3304 - val_loss: 0.3564 - 34ms/epoch - 34ms/step
Epoch 10/300
1/1 - 0s - loss: 0.3292 - val_loss: 0.3552 - 32ms/epoch - 32ms/step
Epoch 11/300
1/1 - 0s - loss: 0.3281 - val_loss: 0.3540 - 30ms/epoch - 30ms/step
Epoch 12/300
1/1 - 0s - loss: 0.3270 - val_loss: 0.3527 - 38ms/epoch - 38ms/step
Epoch 13/300
1/1 - 0s - loss: 0.3258 -

In [32]:
"""Saving model"""
model.save("aus(11).h5")

In [33]:
"""Predictions"""

'Predictions'

In [34]:
pred_aus=model.predict(X_test)

In [35]:
"""Inverse Scaling of the predicted values"""

'Inverse Scaling of the predicted values'

In [36]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]  
    pred_aus[:,:,index]=scaler.inverse_transform(pred_aus[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [37]:
pred_aus = [inverse_difference( pred_aus[i],  rawdatay[i] ) for i in range(len(pred_aus))]
pred_aus=numpy.array(pred_aus)

In [38]:
print(pred_aus.shape)
print(rawdatay.shape)
print('MAPE', (MAPE(rawdatay, pred_aus))[0,:].mean())
'''
print('MAPE', (MAPE(rawdatay, pred_aus))[1,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[2,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[3,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[4,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[5,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[6,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[7,:].mean())

'''

(23, 1, 56)
(23, 1, 56)
MAPE 9.516897363526954


"\nprint('MAPE', (MAPE(rawdatay, pred_aus))[1,:].mean())\n\nprint('MAPE', (MAPE(rawdatay, pred_aus))[2,:].mean())\n\nprint('MAPE', (MAPE(rawdatay, pred_aus))[3,:].mean())\n\nprint('MAPE', (MAPE(rawdatay, pred_aus))[4,:].mean())\n\nprint('MAPE', (MAPE(rawdatay, pred_aus))[5,:].mean())\n\nprint('MAPE', (MAPE(rawdatay, pred_aus))[6,:].mean())\n\nprint('MAPE', (MAPE(rawdatay, pred_aus))[7,:].mean())\n\n"

In [39]:
#################################### """"" LEVEL TWO """"" ############
'''
daily_df=df.iloc[:,56:84]

diff3=daily_df.diff()
#diff3=diff3.values
 
n_past = 1
n_future = 1
n_features = 28

"""Train - Test Split"""

train_df,test_df = diff3[0:12], diff3[12:]  


"""Scaling the values for faster training of the models."""

train = train_df
scalers={}

for i in train_df.columns:

    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s

test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s


"""**Converting the series to samples for supervised learning**"""

def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
where_are_NaNs = isnan(X_train)
X_train[where_are_NaNs] = 0

X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
print(y_train,X_train)


"""Zero Padding"""

z_train=np.zeros((12-n_future,n_past,28))
X_train = np.concatenate([X_train, z_train], -1)
print(X_train.shape)
print(y_train.shape)
z_test=np.zeros((24-n_future,n_past,28))
X_test = np.concatenate([X_test, z_test], -1)
print(X_test.shape)
print(y_test.shape)



rawdataX,rawdatay=split_series(daily_df.values,n_past, n_future)
rawdatay = rawdatay[12:, :]




"""Load the saved model"""

from keras.models import Model
from keras.models import load_model
model2=load_model('aus(1steps).h5')

for layer in model2.layers[:5]:
    layer.trainable = False

# Get input
new_input = model2.input
# Find the layer to connect
hidden_layer = model2.layers[-1].output
# Connect a new layer on it
new_output = Dense(n_features) (hidden_layer)
# Build a new model
new_model2 = Model(new_input, new_output)
new_model2.compile(optimizer='adam', loss='mse')
new_model2.summary()


fit_history2 = new_model2.fit(X_train, y_train, epochs=250, batch_size=50, 
                     validation_split=0.33, verbose=2, shuffle=False) 




"""Predictions"""


pred_aus=new_model2.predict(X_test)
print(pred_aus.shape)



"""Inverse Scaling of the predicted values"""

for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]  
    pred_aus[:,:,index]=scaler.inverse_transform(pred_aus[:,:,index])

   # y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
   # y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

red_aus = [inverse_difference( pred_aus[i],  rawdatay[i] ) for i in range(len(pred_aus))]
pred_aus=numpy.array(pred_aus)
print(pred_aus)
print(pred_aus.shape)



"""Error Estimation"""

print('MAPE', (MAPE(rawdatay, pred_aus))[0,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[0,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[0,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[1,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[1,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[1,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[2,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[2,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[2,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[3,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[3,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[3,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[4,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[4,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[4,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[5,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[5,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[5,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[6,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[6,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[6,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[7,:].mean())

print('RMSE', (RMSE(rawdatay, pred_aus))[7,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[7,:].mean())
#################################### """"" LEVEL One """"" ############
'''
'''
daily_df=df.iloc[:,84:88]

diff3=daily_df.diff()
#diff3=diff3.values
 
n_past = 1
n_future = 8
n_features =4

"""Train - Test Split"""

train_df,test_df = diff3[0:12], diff3[12:] 

"""Scaling the values for faster training of the models."""

train = train_df
scalers={}

for i in train_df.columns:

    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s

test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s


"""**Converting the series to samples for supervised learning**"""

def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
where_are_NaNs = isnan(X_train)
X_train[where_are_NaNs] = 0

X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
print(y_train,X_train)


"""Zero Padding"""

z_train=np.zeros((12-n_future,n_past,52))
X_train = np.concatenate([X_train, z_train], -1)
print(X_train.shape)
print(y_train.shape)
z_test=np.zeros((24-n_future,n_past,52))
X_test = np.concatenate([X_test, z_test], -1)
print(X_test.shape)
print(y_test.shape)



rawdataX,rawdatay=split_series(daily_df.values,n_past, n_future)
rawdatay = rawdatay[12:, :]




from keras.models import Model
from keras.models import load_model
model2=load_model('aus(8steps).h5') #######################

for layer in model2.layers[:5]:
    layer.trainable = False

# Get input
new_input = model2.input
# Find the layer to connect
hidden_layer = model2.layers[-2].output
# Connect a new layer on it
new_output = Dense(n_features) (hidden_layer)
# Build a new model
new_model2 = Model(new_input, new_output)
new_model2.compile(optimizer='adam', loss='mse')
new_model2.summary()


fit_history2 = new_model2.fit(X_train, y_train, epochs=500, batch_size=60, 
                     validation_split=0.33, verbose=2, shuffle=False) 




"""Predictions"""


pred_aus=new_model2.predict(X_test)
print(pred_aus.shape)



"""Inverse Scaling of the predicted values"""

for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]  
    pred_aus[:,:,index]=scaler.inverse_transform(pred_aus[:,:,index])

   # y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
   # y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])







pred_aus = [inverse_difference( pred_aus[i],  rawdatay[i] ) for i in range(len(pred_aus))]
pred_aus=numpy.array(pred_aus)
print(pred_aus)
print(pred_aus.shape)




"""Error Estimation"""
print('MAPE', (MAPE(rawdatay, pred_aus))[0,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[0,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[0,:].mean())



print('MAPE', (MAPE(rawdatay, pred_aus))[1,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[1,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[1,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[2,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[2,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[2,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[3,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[3,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[3,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[4,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[4,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[4,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[5,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[5,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[5,:].mean())


print('MAPE', (MAPE(rawdatay, pred_aus))[6,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[6,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[6,:].mean())

print('MAPE', (MAPE(rawdatay, pred_aus))[7,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[7,:].mean())
print('dRMSE', (dRMSE(rawdatay, pred_aus))[7,:].mean())

'''
#################################### """"" LEVEL One """"" ############
'''
daily_df=df.iloc[:,88:89]

diff3=daily_df.diff()
#diff3=diff3.values
 
n_past = 1
n_future = 8
n_features =1

"""Train - Test Split"""

train_df,test_df = diff3[0:12], diff3[12:] 


"""Scaling the values for faster training of the models."""

train = train_df
scalers={}

for i in train_df.columns:

    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s

test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s


"""**Converting the series to samples for supervised learning**"""

def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
where_are_NaNs = isnan(X_train)
X_train[where_are_NaNs] = 0

X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
print(y_train,X_train)


"""Zero Padding"""

z_train=np.zeros((12-n_future,n_past,55))
X_train = np.concatenate([X_train, z_train], -1)
print(X_train.shape)
print(y_train.shape)
z_test=np.zeros((24-n_future,n_past,55))
X_test = np.concatenate([X_test, z_test], -1)
print(X_test.shape)
print(y_test.shape)


print('kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk', X_train.shape)
rawdataX,rawdatay=split_series(daily_df.values,n_past, n_future)
rawdatay = rawdatay[12:, :]




from keras.models import Model
from keras.models import load_model
model2=load_model('aus(8steps).h5') #######################

for layer in model2.layers[:7]:
    layer.trainable = False

# Get input
new_input = model2.input
# Find the layer to connect
hidden_layer = model2.layers[-2].output
# Connect a new layer on it
new_output = Dense(n_features) (hidden_layer)
# Build a new model
new_model2 = Model(new_input, new_output)
new_model2.compile(optimizer='adam', loss='mse')
new_model2.summary()


fit_history2 = new_model2.fit(X_train, y_train, epochs=450, batch_size=60, 
                     validation_split=0.33, verbose=2, shuffle=False)   #700




"""Predictions"""


pred_aus=new_model2.predict(X_test)
print(pred_aus.shape)



"""Inverse Scaling of the predicted values"""

for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]  
    pred_aus[:,:,index]=scaler.inverse_transform(pred_aus[:,:,index])

   # y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
   # y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])


pred_aus = [inverse_difference( pred_aus[i],  rawdatay[i] ) for i in range(len(pred_aus))]
pred_aus=numpy.array(pred_aus)
print(pred_aus)
print(pred_aus.shape)



def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

dy_true = difference(rawdatay)
dy_pred = difference(pred_aus)



"""Error Estimation"""
print('MAPE', (MAPE(rawdatay, pred_aus))[0,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[0,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[0,:])

print('MAPE', (MAPE(rawdatay, pred_aus))[1,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[1,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[1,:])


print('MAPE', (MAPE(rawdatay, pred_aus))[2,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[2,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[2,:])


print('MAPE', (MAPE(rawdatay, pred_aus))[3,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[3,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[3,:])


print('MAPE', (MAPE(rawdatay, pred_aus))[4,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[4,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[4,:])


print('MAPE', (MAPE(rawdatay, pred_aus))[5,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[5,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[5,:])


print('MAPE', (MAPE(rawdatay, pred_aus))[6,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[6,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[6,:])

print('MAPE', (MAPE(rawdatay, pred_aus))[7,:].mean())
print('RMSE', (RMSE(rawdatay, pred_aus))[7,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[7,:])
'''

'\ndaily_df=df.iloc[:,88:89]\n\ndiff3=daily_df.diff()\n#diff3=diff3.values\n \nn_past = 1\nn_future = 8\nn_features =1\n\n"""Train - Test Split"""\n\ntrain_df,test_df = diff3[0:12], diff3[12:] \n\n\n"""Scaling the values for faster training of the models."""\n\ntrain = train_df\nscalers={}\n\nfor i in train_df.columns:\n\n    scaler = MinMaxScaler(feature_range=(0,1))\n    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))\n    s_s=np.reshape(s_s,len(s_s))\n    scalers[\'scaler_\'+ i] = scaler\n    train[i]=s_s\n\ntest = test_df\nfor i in train_df.columns:\n    scaler = scalers[\'scaler_\'+i]\n    s_s = scaler.transform(test[i].values.reshape(-1,1))\n    s_s=np.reshape(s_s,len(s_s))\n    scalers[\'scaler_\'+i] = scaler\n    test[i]=s_s\n\n\n"""**Converting the series to samples for supervised learning**"""\n\ndef split_series(series, n_past, n_future):\n  #\n  # n_past ==> no of past observations\n  #\n  # n_future ==> no of future observations \n  #\n  X, y = list(), list()\n  